# Lista de exercícios 7 - MNUM-7077

#### Antonio C. da Silva Júnior

Bibliotecas e funções úteis:

In [5]:
import itertools, copy
import numpy as np
import pyomo.environ as pyo
from statistics import *

In [6]:
# Função para declarar um modelo de designação:

def declarar_modelo_designacao(cij):

    # Declaração do modelo:
    modelo = pyo.ConcreteModel()

    # Índices dos pontos:
    modelo.M = pyo.RangeSet(n)
    modelo.N = pyo.RangeSet(n)

    # Matriz de custos:
    modelo.c = pyo.Param(modelo.N, modelo.M, initialize=lambda modelo_D, i, j: cij[i-1][j-1])

    # Variáveis de decisão:
    modelo.x = pyo.Var(modelo.N, modelo.M, within=pyo.Binary)

    # Função objetivo:
    def f_objetivo(modelo):
        return sum(modelo.x[i,j] * modelo.c[i,j] for i in modelo.N for j in modelo.M)

    modelo.objetivo = pyo.Objective(rule=f_objetivo, sense=pyo.minimize)

    # Restrições

    # Cada ponto só recebe de uma única origem:
    def f_restr1(modelo, M):
        return sum(modelo.x[i,M] for i in modelo.N if i!= M) == 1

    modelo.restr1 = pyo.Constraint(modelo.M, rule=f_restr1)

    # Cada ponto só envia para um destino:
    def f_restr2(modelo, N):
        return sum(modelo.x[N,j] for j in modelo.M if j != N) == 1

    modelo.restr2 = pyo.Constraint(modelo.N, rule=f_restr2)
    
    return modelo

In [7]:
# Função que resolve o modelo n vezes e devolve o resultado e um vetor com o tempo de cada execução:

def resolver_modelo(modelo, n_exec=50):
    t = []
    for i in range(n_exec):
        solver = pyo.SolverFactory('glpk')
        result = solver.solve(modelo)
        t.append(result.Solver.Time)
    return t, result

In [8]:
# Função para visualizar os arcos trajeto da solução:

def obter_trajeto(modelo):
    l = list(modelo.x.keys())
    trajeto = []
    for i in l:
        if modelo.x[i]() is not None and modelo.x[i]() != 0:
            trajeto.append(i)
    print(trajeto)
    return trajeto

In [160]:
# Cálcula a média e o desvio padrão do tempo de execução:

def obter_estatisticas(tempo):
    media = round(mean(tempo), 4)
    dp = round(stdev(tempo), 4)
    msg = " Tempo médio: {media} \n Desvio padrão: {dp}".format(media=media, dp=dp)
    print(msg)

In [139]:
# Função para identificar subrotas:

def obter_subrotas(trajeto):
    
    # Ordena o trajeto:
    out = [trajeto[0]]
    while len(out) < n:
        for arco_atual in trajeto:
            ultimo_arco = out[len(out)-1]
            if ultimo_arco[1] == arco_atual[0]:
                if arco_atual not in out:
                    out.append(arco_atual)
        if len(out) < n:
            for proximo_arco in trajeto:
                if proximo_arco not in out:
                    out.append(proximo_arco)

    # Separa as subrotas, caso haja:
    subrota = []
    subrotas = []
    for arco in out:
        if len(subrota) == 0:
            subrota.append(arco)
        else:
            if arco[0] == subrota[len(subrota)-1][1]:
                subrota.append(arco)
            else:
                subrotas.append(subrota)
                subrota = []
                subrota.append(arco)
    subrotas.append(subrota)
    
    if(len(subrotas) > 1):
        msg = "Há {n} subrotas!".format(n=len(subrotas))
    else:
        msg = "Não há subrotas!"
        
    print(msg)
    print(subrotas)
        
    return subrotas

In [140]:
# Função para calcular o custo das subrotas

def obter_custo_subrotas(subrotas):
    dict_subrotas = {}
    for i in range(len(subrotas)):
        sub = subrotas[i]
        custo = 0
        for arco in sub:
            custo += cij[arco[0]-1][arco[1]-1]
        dict_subrotas[i] = {'subrota': sub,
                            'custo': round(custo,2),}

    print(dict_subrotas)

## 1 - Exercício A (10 pontos)

Matriz de custos:

In [141]:
cij = [[0,    49,   46.6, 80.8, 67.2, 35.4, 68.7, 53.6, 57.8, 73.9],
       [49,   0,    59.8, 37,   37.1, 23.4, 70.2, 48.1, 27,   41.8],
       [46.6, 59.8, 0,    70.2, 47.3, 64.8, 23,   20.1, 83.2, 52.6],
       [80.8, 37,   70.2, 0,    23.9, 60.4, 68.2, 51,   57.3, 21.3],
       [67.2, 37.1, 47.3, 23.9, 0,    57.7, 44.6, 27.5, 63.5, 6.7],
       [35.4, 23.4, 64.8, 60.4, 57.7, 0,    81.2, 60,   22.5, 63.3],
       [68.7, 70.2, 23,   68.2, 44.6, 81.2, 0,    22.1, 96.3, 47.4],
       [53.6, 48.1, 20.1, 51,   27.5, 60,   22.1, 0,    74.2, 32.6],
       [57.8, 27,   83.2, 57.3, 63.5, 22.5, 96.3, 74.2, 0,    67.5],
       [73.9, 41.8, 52.6, 21.3, 6.7,  63.3, 47.4, 32.6, 67.5, 0]]

cij = [[0,    49,   46.6, 80.8],
       [49,   0,    59.8, 37],
       [46.6, 59.8, 0,    70.2],
       [80.8, 37,   70.2, 0]]

###  1.1 - Modelo com restrições de sub-rota de Dantzig

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$\sum\limits_{i \in Q} \sum\limits_{j \notin Q} x_{ij} \geq 1 \;\;\; \forall \; Q \subseteq \{1,...,n\}, 1 \leq |Q| \leq n-1$

$x_{ij} \in \{0,1\}$


#### 1.1.1 - Resolução

In [142]:
# Número de pontos:
n = len(cij)
print(n)

4


In [143]:
# Lógica para inclusão dinâmica das restrições de sub-rota de Dantzig:

# Combinações:

c = []
for i in range(n):
    c.append([0,1])
    
comb = np.array(list(itertools.product(*c)))
comb = np.flip(comb, axis=1)
comb = comb[(np.sum(comb, axis=1) > 1) & (np.sum(comb, axis=1) < (n-1))] # remove todos os vazis, cheios e redundantes
comb

array([[1, 1, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 1]])

In [144]:
# Definição dos conjuntos Q e não Q:
Q = []
nQ = []
for i in range(len(comb)):
    q = []
    n_q = []
    for j in range(len(comb[i])):
        if comb[i,j] == 1:
            q.append(j+1)
        else:
            n_q.append(j+1)
    Q.append(q)
    nQ.append(n_q)
    
print(Q)

[[1, 2], [1, 3], [2, 3], [1, 4], [2, 4], [3, 4]]


In [145]:
print(nQ)

[[3, 4], [2, 4], [1, 4], [2, 3], [1, 3], [1, 2]]


In [146]:
# Declaração do modelo:
modelo_D = declarar_modelo_designacao(cij)

# Inclusão das restrições de sub-rotas de Dantzig:
modelo_D.restr_dantzig = pyo.ConstraintList()

for q, nq in zip(Q, nQ):
    modelo_D.restr_dantzig.add(sum(modelo_D.x[i,j] for i in q for j in nq) >= 1)

In [147]:
# Resolução:

# 50 execuções para obter o tempo médio:
tempo_D, resultado_D = resolver_modelo(modelo_D)
print(resultado_D)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 13
  Number of nonzeros: 49
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.035904884338378906
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [148]:
trajeto_D = obter_trajeto(modelo_D)

[(1, 2), (2, 4), (3, 1), (4, 3)]


In [149]:
obter_estatisticas(tempo_D)

Tempo médio: 0.039 
 Desvio padrão: 0.0067


###  1.2 - Modelo com restrições de sub-rota MTZ

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

$u_i = \text{Variável auxiliar para definição das restrições de sub-rota MTZ} \;\;\; \forall \; i=1,...,m$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$u_1 = 1$

$u_i - u_j + nx_{ij} \leq n-1 \;\;\; \forall \; i,j \in \{2,...,n\}, i \neq j$

$x_{ij} \in \{0,1\}$

$u_i \geq 0$

#### 1.2.1 - Resolução

In [150]:
# Número de pontos:
n = len(cij)
print(n)

4


In [151]:
# Declaração do modelo:
modelo_MTZ = declarar_modelo_designacao(cij)

# Índice para a variável auxiliar u:
modelo_MTZ.U = pyo.RangeSet(2,n)

# Váriavel auxiliar u:
modelo_MTZ.u = pyo.Var(modelo_MTZ.N, within=pyo.NonNegativeIntegers,bounds=(0,n-1))

# Restrições de sub-rota MTZ:
def f_MTZ(modelo, i, j):
    if i!=j: 
        return modelo.u[i] - modelo.u[j] + modelo.x[i,j] * n <= n-1
    else:
        return modelo.u[i] - modelo.u[i] == 0 # sem efeito no modelo
    
modelo_MTZ.restr_MTZ = pyo.Constraint(modelo_MTZ.U, modelo_MTZ.N, rule=f_MTZ)

In [152]:
# Resolução:

# 50 execuções para obter o tempo médio:
tempo_MTZ, resultado_MTZ = resolver_modelo(modelo_MTZ)
print(resultado_MTZ)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 21
  Number of variables: 17
  Number of nonzeros: 52
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.0359034538269043
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [153]:
trajeto_MTZ = obter_trajeto(modelo_MTZ)

[(1, 2), (2, 4), (3, 1), (4, 3)]


In [154]:
obter_estatisticas(tempo_MTZ)

Tempo médio: 0.0383 
 Desvio padrão: 0.0018


###  1.3 - Modelo através da variante do branch and bound

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$x_{ij} \in \{0,1\}$


#### 1.3.1 - Resolução do modelo de designação

In [156]:
# Número de pontos:
n = len(cij)
print(n)

4


In [157]:
# Declaração do modelo de designação:
modelo_B = declarar_modelo_designacao(cij)

# Resolução:
tempo_B, resultado_B = resolver_modelo(modelo_B, 50)
print(resultado_B)


Problem: 
- Name: unknown
  Lower bound: 167.2
  Upper bound: 167.2
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.0359041690826416
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [162]:
obter_estatisticas(tempo_B)

 Tempo médio: 0.037 
 Desvio padrão: 0.0014


In [163]:
trajeto_B = obter_trajeto(modelo_B)
print("")
subrotas_B = obter_subrotas(trajeto_B)
print("")
custo_subrotas_B = obter_custo_subrotas(subrotas_B)

[(1, 3), (2, 4), (3, 1), (4, 2)]

Há 2 subrotas!
[[(1, 3), (3, 1)], [(2, 4), (4, 2)]]

{0: {'subrota': [(1, 3), (3, 1)], 'custo': 93.2}, 1: {'subrota': [(2, 4), (4, 2)], 'custo': 74}}


Necessário resolver os seguintes problemas:

- <b>Problema 00:</b>

    -$c_{13}=9999$
        
<br>
    
- <b>Problema 01:</b>

    -$c_{31}=9999$
    
<br>
    
- <b>Problema 02:</b>

    -$c_{24}=9999$
    
    
<br>
    
- <b>Problema 03:</b>

    -$c_{42}=9999$

#### 1.3.2 - Início do método branch and bound

##### 1.3.2.1 - Problema 00

In [164]:
# Atualização da matriz de custos:
cij_00 = copy.deepcopy(cij)
cij_00[0][2] = 999999 # [1,3]

cij_00

[[0, 49, 999999, 80.8],
 [49, 0, 59.8, 37],
 [46.6, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [165]:
# Declaração do modelo:
modelo_B_00 = declarar_modelo_designacao(cij_00)

# Resolução
tempo_B_00, resultado_B_00 = resolver_modelo(modelo_B_00, 50)
print(resultado_B_00)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.036904335021972656
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [166]:
obter_estatisticas(tempo_B_00)

 Tempo médio: 0.0386 
 Desvio padrão: 0.0021


In [167]:
ttrajeto_B_00 = obter_trajeto(modelo_B_00)
print("")
subrotas_B_00 = obter_subrotas(trajeto_B_00)
print("")
custo_subrotas_B_00 = obter_custo_subrotas(subrotas_B_00)

[(1, 2), (2, 4), (3, 1), (4, 3)]

Não há subrotas!
[[(1, 2), (2, 4), (4, 3), (3, 1)]]

{0: {'subrota': [(1, 2), (2, 4), (4, 3), (3, 1)], 'custo': 202.8}}


Encontrado um <b>bound</b>!

##### 1.3.2.2 - Problema 01

In [168]:
# Atualização da matriz de custos:
cij_01 = copy.deepcopy(cij)
cij_01[2][0] = 999999 # [3,1]

cij_01

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 37],
 [999999, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [169]:
# Declaração do modelo:
modelo_B_01 = declarar_modelo_designacao(cij_01)

# Resolução
tempo_B_01, resultado_B_01 = resolver_modelo(modelo_B_01, 50)
print(resultado_B_01)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04089236259460449
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [171]:
obter_estatisticas(tempo_B_01)

 Tempo médio: 0.0385 
 Desvio padrão: 0.002


In [172]:
ttrajeto_B_01 = obter_trajeto(modelo_B_01)
print("")
subrotas_B_01 = obter_subrotas(trajeto_B_01)
print("")
custo_subrotas_B_01 = obter_custo_subrotas(subrotas_B_01)

[(1, 3), (2, 1), (3, 4), (4, 2)]

Não há subrotas!
[[(1, 3), (3, 4), (4, 2), (2, 1)]]

{0: {'subrota': [(1, 3), (3, 4), (4, 2), (2, 1)], 'custo': 202.8}}


Encontrado um <b>bound</b>!

##### 1.3.2.3 - Problema 02

In [173]:
# Atualização da matriz de custos:
cij_02 = copy.deepcopy(cij)
cij_02[1][3] = 999999 # [2,4]

cij_02

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 999999],
 [46.6, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [174]:
# Declaração do modelo:
modelo_B_02 = declarar_modelo_designacao(cij_02)

# Resolução
tempo_B_02, resultado_B_02 = resolver_modelo(modelo_B_02, 50)
print(resultado_B_02)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.036898136138916016
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [175]:
obter_estatisticas(tempo_B_02)

 Tempo médio: 0.0381 
 Desvio padrão: 0.0028


In [176]:
ttrajeto_B_02 = obter_trajeto(modelo_B_02)
print("")
subrotas_B_02 = obter_subrotas(trajeto_B_02)
print("")
custo_subrotas_B_02 = obter_custo_subrotas(subrotas_B_02)

[(1, 3), (2, 1), (3, 4), (4, 2)]

Não há subrotas!
[[(1, 3), (3, 4), (4, 2), (2, 1)]]

{0: {'subrota': [(1, 3), (3, 4), (4, 2), (2, 1)], 'custo': 202.8}}


Encontrado um <b>bound</b>!

##### 1.3.2.3 - Problema 02

In [177]:
# Atualização da matriz de custos:
cij_03 = copy.deepcopy(cij)
cij_03[3][1] = 999999 # [4,2]

cij_03

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 37],
 [46.6, 59.8, 0, 70.2],
 [80.8, 999999, 70.2, 0]]

In [178]:
# Declaração do modelo:
modelo_B_03 = declarar_modelo_designacao(cij_03)

# Resolução
tempo_B_03, resultado_B_03 = resolver_modelo(modelo_B_03, 50)
print(resultado_B_03)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03789925575256348
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [179]:
obter_estatisticas(tempo_B_03)

 Tempo médio: 0.0378 
 Desvio padrão: 0.0022


In [181]:
trajeto_B_03 = obter_trajeto(modelo_B_03)
print("")
subrotas_B_03 = obter_subrotas(trajeto_B_03)
print("")
custo_subrotas_B_03 = obter_custo_subrotas(subrotas_B_03)

[(1, 2), (2, 4), (3, 1), (4, 3)]

Não há subrotas!
[[(1, 2), (2, 4), (4, 3), (3, 1)]]

{0: {'subrota': [(1, 2), (2, 4), (4, 3), (3, 1)], 'custo': 202.8}}


Encontrado um <b>bound</b>!